# Brain Imaging Data Structure Converter 

Converts the Net Station data into a single session of a single subject. Iterate to create a BIDS compliant data structure, Check with the BIDS validator to see if this data structure is valid.

### Import ✔️

In [1]:
import matplotlib
import pathlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

### Create a Directory ✔️

Rewrite this subject directory object to connect to local PCs raw lab data location.

In [2]:
subject_dir = "C:/Users/trevo/Desktop/Lab_Files/WM_eegData/"
subject_dir = pathlib.Path(subject_dir)

Rewrite this below code each time as new data is read in.

In [3]:
raw_path = subject_dir / 'WM1' / 'S1' / 'WM_1_isi0_s1c_20191110_020011.mff'
raw = mne.io.read_raw_egi(raw_path)

Reading EGI MFF Header from C:\Users\trevo\Desktop\Lab_Files\WM_eegData\WM1\S1\WM_1_isi0_s1c_20191110_020011.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {SESS} ...


### Find and Declare Event Names ✔️

This will label all trial_type information with these event names, found in sub-01_ses-01_task-WM1 isi0_events.tsv within the session data.

In [4]:
pre_events = mne.find_events(raw, 'STI 014')
events = mne.pick_events(pre_events, exclude=[1, 3])
event_id = {
    'CELL': 1,
    'bgin': 2,
    'TRSP': 3,
    'fix+': 4,
    'WMD+': 5,
    'TsD-': 6
}

events #Figure out these events!

533 events found
Event IDs: [1 2 3 4 5 6]


array([[  4104,      0,      2],
       [  5637,      0,      4],
       [  6137,      0,      5],
       ...,
       [359901,      0,      4],
       [360401,      0,      5],
       [360869,      0,      6]], dtype=int64)

### Specify the Power Line Frequency ✔️

In [5]:
raw.info['line_freq'] = 60 #The hertz of the powerlines fed into the lab 

### Specify Subject Information ✔️

You will need to edit this as each new raw data input is made, look for way to streamline/automate this? Figure out a way to put initial information to participants.tsv file

In [6]:
subject_info = {
    'initials' : 'DAT'
    #'birthday': (1970,10,1),
    #'sex': 2,
    #'hand': 2
}

raw.info['subject_info'] = subject_info
raw.info

<Info | 12 non-empty values
 bads: []
 ch_names: E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11, E12, E13, E14, ...
 chs: 129 EEG, 8 Stimulus
 custom_ref_applied: False
 device_info: 1 item (dict)
 dig: 133 items (3 Cardinal, 130 EEG)
 highpass: 0.0 Hz
 line_freq: 60.0
 lowpass: 500.0 Hz
 meas_date: 2019-11-10 20:00:11 UTC
 nchan: 137
 projs: []
 sfreq: 1000.0 Hz
 subject_info: 1 item (dict)
 utc_offset: -0600
>

### Write in the Data ❌

Will list this data in participants.json and participants.tsv, as well as use it to establish the direction of the data structure.

You will need to specify the subject number, session, and task on each iteration.

In [7]:
out_path = pathlib.Path('out_data/bids_structure') #Into which directory the data should go

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='WM1isi0',
                              root=out_path) #An object that specifies the location of BIDS files

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events, format="BrainVision",
                        event_id=event_id) #Writes the data

Reading EGI MFF Header from C:\Users\trevo\Desktop\Lab_Files\WM_eegData\WM1\S1\WM_1_isi0_s1c_20191110_020011.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {SESS} ...
Writing 'out_data\BIDS_Structure\participants.tsv'...
Writing 'out_data\BIDS_Structure\participants.json'...
Writing 'out_data/BIDS_Structure/sub-01/ses-01/eeg/sub-01_ses-01_space-CapTrak_electrodes.tsv'...
Writing 'out_data/BIDS_Structure/sub-01/ses-01/eeg/sub-01_ses-01_space-CapTrak_coordsystem.json'...
Used Annotations descriptions: ['TsD-', 'WMD+', 'bgin', 'fix+']
Writing 'out_data\BIDS_Structure\sub-01\ses-01\eeg\sub-01_ses-01_task-WM1isi0_events.tsv'...
Writing 'out_data\BIDS_Structure\dataset_description.json'...
Writing 'out_data\BIDS_Structure\sub-01\ses-01\eeg\sub-01_ses-01_task-WM1isi0_eeg.json'...
Writing 'out_data\BIDS_Structure\sub-01\ses-01\eeg\sub-01_ses-01_task-WM1isi0_channels.tsv'...
Copying data files to sub-01_ses-0

C:\Users\trevo\AppData\Local\Temp\ipykernel_14892\1018502517.py:8: RuntimeWarning: Converting data files to BrainVision format
  mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events, format="BrainVision",
C:\Users\trevo\AppData\Local\Temp\ipykernel_14892\1018502517.py:8: RuntimeWarning: Encountered data in "float" format. Converting to float32.
  mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events, format="BrainVision",
C:\Users\trevo\anaconda3\envs\mne\lib\site-packages\pybv\io.py:603: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


Writing 'out_data\BIDS_Structure\sub-01\ses-01\sub-01_ses-01_scans.tsv'...
Wrote out_data\BIDS_Structure\sub-01\ses-01\sub-01_ses-01_scans.tsv entry with eeg\sub-01_ses-01_task-WM1isi0_eeg.vhdr.


BIDSPath(
root: out_data/BIDS_Structure
datatype: eeg
basename: sub-01_ses-01_task-WM1isi0_eeg.vhdr)